In [ ]:
!conda install pytorch torchvision torchaudio -c pytorch -y

In [1]:
# import openCv
import cv2
# import uuid
import uuid
# import operating system
import os
# import time
import time
from ultralytics import YOLO
# ignore warnings
from warnings import filterwarnings
filterwarnings("ignore")

# 2. Define images to collect

In [ ]:
labels = ['thumbsup', 'thumbsdown', 'thankyou', 'livelong']
labels = ['thankyou']
number_img = 5

In [ ]:
print(number_img)

# 3. Setup Folders

In [11]:
Images_paths = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

In [36]:
print(Images_paths)

Tensorflow/workspace/images/collectedimages


In [13]:
if not os.path.exists(Images_paths):
    if os.name =='posix':
        os.makedirs(Images_paths, exist_ok=True)
    if os.name == 'nt':
        os.makedirs(Images_paths, exist_ok=True)
for label in labels:
    path = os.path.join(Images_paths, label)
    if not os.path.exists(path):
        os.makedirs (path, exist_ok=True)

# 4. Capture Images

In [15]:
for label in labels:
    cap = cv2.VideoCapture(0) # connects to the webcap or capture device
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_img):
        print('collecting images {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(Images_paths, label, label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname,frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

[ WARN:0@1285.065] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


collecting images 0
collecting images 1
collecting images 2
collecting images 3
collecting images 4


# Data training and testing

In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
# BASE_PATH = os.path.expanduser("~/Documents/programmingLanguages/PythonProgramming/ObjectDetection/Tensorflow")  # goes to your Documents folder
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
files["PIPELINE_CONFIG"]

In [6]:
# split the images 
import os, random, shutil

base_dir = "Tensorflow"
image_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# output dirs
for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(base_dir, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, split, 'labels'), exist_ok=True)

# get all images
images = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.png')]
random.shuffle(images)

# split sizes
train_split = int(0.75 * len(images))
val_split = int(0.95 * len(images))

splits = {
    'train': images[:train_split],
    'val': images[train_split:val_split],
    'test': images[val_split:]
}

# move files
for split, imgs in splits.items():
    for img in imgs:
        label = img.rsplit('.', 1)[0] + '.txt'
        shutil.copy(os.path.join(image_dir, img), os.path.join(base_dir, split, 'images', img))
        shutil.copy(os.path.join(labels_dir, label), os.path.join(base_dir, split, 'labels', label))

print("Dataset split complete.")

Dataset split complete.


In [17]:
model = YOLO("yolov8s.pt")
result = model.train(
    data="/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/ObjectDetection/Tensorflow/dataset.yaml",
    epochs=50,
    imgsz=640,
    batch=16
)

Ultralytics 8.3.209 🚀 Python-3.10.18 torch-2.2.2 CPU (Intel Core i5-5350U 1.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/ObjectDetection/Tensorflow/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=None, optimize=False, opti

RuntimeError: No valid images found in /Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/ObjectDetection/Tensorflow/val/labels.cache. Images with incorrectly formatted labels are ignored. See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [15]:
import glob

label_paths = glob.glob("/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/ObjectDetection/Tensorflow/train/labels/*.txt")
classes = set()

for path in label_paths:
    with open(path) as f:
        for line in f:
            if line.strip():
                cls_id = int(line.split()[0])
                classes.add(cls_id)

print("Classes found in train:", classes)


Classes found in train: {0, 1, 2, 3}


In [11]:
import glob

label_dir = "/Users/briankimanzi/Documents/programmingLanguages/PythonProgramming/ObjectDetection/Tensorflow/train/labels"

mapping = {15: 0, 16: 1, 17: 2, 18: 3}  # adjust based on your actual class meaning

for label_path in glob.glob(f"{label_dir}/*.txt"):
    lines = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                cls_id = int(parts[0])
                if cls_id in mapping:
                    parts[0] = str(mapping[cls_id])
                    lines.append(" ".join(parts))
    with open(label_path, "w") as f:
        f.write("\n".join(lines))


<h2>Download TF Models Pretrained Models from tensorflow Model Zoo and Install TFOD</h2>